Please open https://sandbox.neo4j.com/

For query all the nodes `MATCH(n) RETURN(n)`

# Install

In [2]:
!pip install -q neo4j-driver --use-deprecated=legacy-resolver
!pip install -q gradio --use-deprecated=legacy-resolver

# Imports

In [3]:
from neo4j import GraphDatabase
import spacy
import gradio as gr

In [4]:
driver = GraphDatabase.driver("bolt://44.192.118.201:7687",
                              auth=("neo4j",
                                    "slides-seesaws-checkouts"))

In [5]:
nlp = spacy.load("en_core_web_sm")

# Functions

In [6]:
def extract_entity(question, entity_type):
    doc = nlp(question)
    for ent in doc.ents:
        if ent.label_ in entity_type:
            return ent.text
    return None

In [7]:
def chatbot(input, history=[]):
    output = get_answer(input)
    history.append((input, output))
    return history, history

In [8]:
def get_answer(question):
    doc = nlp(question)
    movie_title = extract_entity(question, ["WORK_OF_ART", "MOVIE"])
    director_name = extract_entity(question, ["PERSON"])
    if movie_title is None:
        return "Sorry, I don't understand what movie you're asking about."

    with driver.session() as session:
        if "release" in question or "released" in question:
            result = session.run(f"MATCH (m:Movie) WHERE m.title = '{movie_title}' RETURN m.released")
            if result.peek() is None:
                return f"No information found for the movie '{movie_title}'"
            for record in result:
                return f"The release date of '{movie_title}' is: {record['m.released']}"
        else:
            return "Sorry, I don't understand what you're asking."

# Interface

In [9]:
gr.Interface(fn = chatbot,
             inputs = ["text",'state'],
             outputs = ["chatbot",'state']).launch(debug = True)

Setting queue=True in a Colab notebook requires sharing enabled. Setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. This cell will run indefinitely so that you can see errors and logs. To turn off, set debug=False in launch().
Running on public URL: https://ef7b0c2ce859760524.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from Terminal to deploy to Spaces (https://huggingface.co/spaces)


Keyboard interruption in main thread... closing server.
Killing tunnel 127.0.0.1:7860 <> https://ef7b0c2ce859760524.gradio.live
